In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

%matplotlib inline

Merging Waves

In [ ]:
wave1 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID'])
wave3educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID'])
wave3grad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0017\21600-0017-Data.tsv", sep = '\t', usecols = ['AID'])
wave4 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID'])

In [ ]:
waves = [wave1, wave3educ, wave3grad, wave4]
AID = reduce(lambda left, right: pd.merge(left, right, on = 'AID', how = 'inner'), waves)
len(AID)

GPA

In [ ]:
hs_gpa = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID', 'EAOGPAC'])
hs_gpa = hs_gpa.rename(columns = {'AID': 'AID', 'EAOGPAC': 'cumulative_gpa'})
hs_gpa = pd.merge(AID, hs_gpa, on = 'AID', how = 'left')
#hs_gpa.isna().sum() #check for nulls to make sure the AID matched
bins = [-1, 1, 2, 3 ,4]
labels = ['0-1', '1-2', '2-3', '3-4']
hs_gpa['gpa_range'] = pd.cut(np.array(hs_gpa['cumulative_gpa']), bins = bins, labels = labels)
hs_gpa = hs_gpa.drop(hs_gpa[hs_gpa['cumulative_gpa'] == 9995].index)
hs_gpa

Grades -- Wave 1

In [ ]:
grades = pd.read_csv(r"\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1ED11', 'H1ED12', 'H1ED13', 'H1ED14'], low_memory = False)
grades = grades.rename(columns = {'AID': 'AID','H1ED11': 'English', 'H1ED12' : 'Math', 'H1ED13' : 'History', 'H1ED14' : 'Science'})
grades

In [ ]:
grades[['English', 'Math', 'History', 'Science']].corr()

Academic Attainment

Peers

In [ ]:
peers = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1ED18','H1ED19', 'H1ED24'])
peers = peers.rename(columns = {'H1ED18':'getting_along','H1ED19':'close_peers','H1ED24':'safe_school'})
peers = pd.merge(AID, peers, how = 'left', on = 'AID')
#peers.isna().sum()
#peers['getting_along'].value_counts() # get rid of 7 (57 ppl)
#peers['close_peers'].value_counts() #get rid of 7 (57 ppl) and 8 (1)
#len(peers.loc[peers['getting_along'] == 7])#= 57, same 57 state 7
#peers.loc[peers['close_peers'] == 8] # keep this value b/c they have answers for the other columns
peers = peers.drop(peers[peers['getting_along'] == 7].index)
peers

In [ ]:
peers = pd.merge(peers, hs_gpa, on = 'AID', how = 'inner')
len(peers)

In [ ]:
peers[['getting_along', 'close_peers', 'safe_school', 'cumulative_gpa']].corr()

In [ ]:
peer_grades = pd.merge(peers, grades, on = 'AID', how = 'inner')
peer_grades

In [ ]:
peer_grades[['getting_along', 'close_peers', 'safe_school','English', 'Math', 'History', 'Science']].corr()

In [ ]:
peer_grades[['English', 'Math', 'History', 'Science', 'cumulative_gpa']].corr()

Friends

In [ ]:
friends = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1PR4'])
friends = friends.rename(columns = {'H1PR4': 'caring_friends'})
friends = pd.merge(AID, friends, how = 'left', on = 'AID')
#friends.isna().sum()
#friends['caring_friends'].value_counts() #drop 6, 96, 98
friends = friends.drop(friends[(friends['caring_friends'] == 6) |(friends['caring_friends'] == 96) | (friends['caring_friends'] == 98)].index)
friends

In [ ]:
friends = pd.merge(friends, hs_gpa, on = 'AID', how = 'inner')
len(friends)

In [ ]:
friends[['caring_friends', 'cumulative_gpa']].corr()

In [ ]:
friends_grades = pd.merge(friends, grades, on = 'AID', how = 'inner')
friends_grades

In [ ]:
friends_grades[['caring_friends', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
friends_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

Mom -- Relationship

In [ ]:
mom = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID','H1WP9', 'H1WP10'])
mom = mom.rename(columns = {'H1WP9': 'mom_close', 'H1WP10': 'caring_mom'})
mom = pd.merge(AID, mom, how = 'left', on = 'AID')
mom.isna().sum()
mom['mom_close'].value_counts() #drop 8 and 6 (2 each) and 7 (148)
mom['caring_mom'].value_counts() #drop 8 and 6 if overlap (1 each) and 7 (148)
mom.loc[(mom['mom_close'] == 6) | (mom['mom_close'] == 8)]
mom = mom.drop(mom[mom['mom_close'] == 7].index)
mom = mom.drop([1097, 3054])
mom

In [ ]:
mom = pd.merge(mom, hs_gpa, how = 'inner', on = 'AID')
mom

In [ ]:
mom[['mom_close', 'caring_mom', 'cumulative_gpa']].corr()

In [ ]:
mom_grades = pd.merge(mom, grades, on = 'AID', how = 'inner')
mom_grades

In [ ]:
mom_grades[['mom_close', 'caring_mom', 'English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

Mom -- View on School -- Disappointed if they don't graduate college/hs 

In [ ]:
mom_school = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP11', 'H1WP12'])
mom_school = mom_school.rename(columns = {'H1WP11': 'graduate_college', 'H1WP12': 'graduate_hs'})
mom_school = pd.merge(AID, mom_school, on = 'AID', how = 'left')
mom_school.isna().sum()
mom_school['graduate_college'].value_counts() #drop 6(1) and 8(7) and 7 (148)
mom_school['graduate_hs'].value_counts() #drop 6 (2), 8 (7), 9(1) that overlap and 7 (148)
mom_school.loc[(mom_school['graduate_hs'] == 6) | (mom_school['graduate_hs'] == 8) | (mom_school['graduate_hs'] == 9)]
mom_school = mom_school.drop([1097, 2736, 2892, 3054])
mom_school = mom_school.drop(mom_school[mom_school['graduate_hs'] == 7].index)
mom_school

In [ ]:
mom_school = pd.merge(mom_school, hs_gpa, on = 'AID', how = 'inner')
mom_school

In [ ]:
mom_school[['graduate_college', 'graduate_hs', 'cumulative_gpa']].corr()

In [ ]:
mom_school_grades = pd.merge(mom_school, grades, on = 'AID', how = 'inner')
mom_school_grades

In [ ]:
mom_school_grades[['graduate_college', 'graduate_hs','English', 'Math', 'History', 'Science']].corr()

In [ ]:
mom_school_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

Dad -- Relationship

In [ ]:
#significantly more kids who don't have dads (value 7) than who don't have moms
dad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP13', 'H1WP14'])
dad = dad.rename(columns = {'H1WP13':'dad_close', 'H1WP14':'caring_dad'})
dad = pd.merge(AID, dad, how = 'left', on = 'AID')
dad.isna().sum()
dad['dad_close'].value_counts() #drop 6(1), 7 (904)
dad['caring_dad'].value_counts() #drop 6(1), 7 (904)
dad = dad.drop(dad[(dad['dad_close'] == 7) | (dad['dad_close'] == 6)].index)
dad

In [ ]:
dad = pd.merge(dad, hs_gpa, on = 'AID', how = 'inner')
dad

In [ ]:
dad[['dad_close', 'caring_dad', 'cumulative_gpa']].corr()

In [ ]:
dad_grades = pd.merge(dad, grades, on = 'AID', how = 'inner')
dad_grades

In [ ]:
dad_grades[['dad_close', 'caring_dad','English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_grades[['cumulative_gpa', 'English', 'Math', 'History', 'Science']].corr()

Dad -- View on School -- Disappointed if they don't graduate college/hs

In [ ]:
dad_school = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1WP15', 'H1WP16'])
dad_school = dad_school.rename(columns = {'H1WP15': 'graduate_college', 'H1WP16': 'graduate_hs'})
dad_school = pd.merge(AID, dad_school, on = 'AID', how = 'left')
dad_school.isna().sum()
dad_school['graduate_college'].value_counts() #drop 7 (904), 8(4), 6(1)
dad_school['graduate_hs'].value_counts() #drop 7(904), 8(3), 6(4)
dad_school = dad_school.drop(dad_school[dad_school['graduate_hs'] == 7].index)
dad_school.loc[(dad_school['graduate_hs'] == 8) | (dad_school['graduate_hs'] == 6)]
dad_school = dad_school.drop([1097, 1816, 3163])

In [ ]:
dad_school = pd.merge(dad_school, hs_gpa, on = 'AID', how = 'inner')
dad_school

In [ ]:
dad_school[['graduate_college', 'graduate_hs', 'cumulative_gpa']].corr()

In [ ]:
dad_school_grades = pd.merge(dad_school, grades, on = 'AID', how = 'inner')
dad_school_grades

In [ ]:
dad_school_grades[['graduate_college', 'graduate_hs','English', 'Math', 'History', 'Science']].corr()

In [ ]:
dad_school_grades[['cumulative_gpa','English', 'Math', 'History', 'Science']].corr()

Kids w/ no mom or dad figure